<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bs4 as bs
import urllib
import re

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [25]:
#Selecionar las paginas de Wikipedia para descargar
pages = [
    "https://en.wikipedia.org/wiki/NASA",
    'https://en.wikipedia.org/wiki/SpaceX',
    "https://en.wikipedia.org/wiki/SpaceX_Starship",
    "https://en.wikipedia.org/wiki/Ariane_5",
    "https://en.wikipedia.org/wiki/Space_Shuttle",
    "https://en.wikipedia.org/wiki/Saturn_V",
    "https://en.wikipedia.org/wiki/Delta_IV",
    "https://en.wikipedia.org/wiki/Atlas_V",
    "https://en.wikipedia.org/wiki/Vulcan_Centaur",
    "https://en.wikipedia.org/wiki/Falcon_9",
    "https://en.wikipedia.org/wiki/Boeing_777",
    "https://en.wikipedia.org/wiki/Boeing_787_Dreamliner",
    "https://en.wikipedia.org/wiki/Boeing_737",
    "https://en.wikipedia.org/wiki/Airbus_A320_family",
    "https://en.wikipedia.org/wiki/Airbus_A330neo",
    "https://en.wikipedia.org/wiki/Airbus_A350",
    "https://en.wikipedia.org/wiki/Airbus_A330",
    "https://en.wikipedia.org/wiki/Boeing_747",
    "https://en.wikipedia.org/wiki/Boeing_767"
]

In [26]:
# Crear una funcion para obtener las paginas

def get_page_text(url):
    raw_html = urllib.request.urlopen(url)
    raw_html = raw_html.read()

    article_html = bs.BeautifulSoup(raw_html, 'lxml')

    article_paragraphs = article_html.find_all('p')

    #Remover las referencias en las líneas, ej. "[23]"
    text = [re.sub(r"\[([0-9]*)\]", '', p.text) for p in article_paragraphs]

    return text


In [27]:
#Obtener todas las lineas
lineas = []

for page in pages:
    text = get_page_text(page)
    print(len(text))
    lineas += text

len(lineas)

211
92
86
47
70
81
65
63
37
89
96
141
102
94
62
103
101
100
85


1725

### 1 - Preprocesamiento

In [29]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for row in lineas:
    #print(row[0])
    sentence_tokens.append(text_to_word_sequence(row))

In [30]:
# Demos un vistazo
sentence_tokens[:2]

[[],
 ['the',
  'national',
  'aeronautics',
  'and',
  'space',
  'administration',
  'nasa',
  'ˈnæsə',
  'is',
  'an',
  'independent',
  'agency',
  'of',
  'the',
  'u',
  's',
  'federal',
  'government',
  'responsible',
  'for',
  'the',
  'civil',
  'space',
  'program',
  'aeronautics',
  'research',
  'and',
  'space',
  'research']]

### 2 - Crear los vectores (word2vec)

In [31]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [32]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores, "vector_size" reemplazando "size"
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [33]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [34]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1725


In [35]:
# Cantidad de words encontradas en el corpus
#print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 3011


### 3 - Entrenar el modelo generador

In [36]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1320446.75
Loss after epoch 1: 948503.0
Loss after epoch 2: 840321.0
Loss after epoch 3: 821755.0
Loss after epoch 4: 775953.75
Loss after epoch 5: 751022.0
Loss after epoch 6: 740449.0
Loss after epoch 7: 731173.5
Loss after epoch 8: 722658.0
Loss after epoch 9: 710738.0
Loss after epoch 10: 675477.0
Loss after epoch 11: 667950.0
Loss after epoch 12: 660111.0
Loss after epoch 13: 656760.0
Loss after epoch 14: 651020.0
Loss after epoch 15: 647770.0
Loss after epoch 16: 643328.0
Loss after epoch 17: 642376.0
Loss after epoch 18: 642378.0
Loss after epoch 19: 642029.0


(1930474, 2806760)

### 4 - Ensayar

In [37]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rocket"], topn=10)

[('motors', 0.5478342175483704),
 ('boosters', 0.5419757962226868),
 ('strap', 0.5229859352111816),
 ('ii', 0.5092114210128784),
 ('srb', 0.4951074421405792),
 ('\u200aii', 0.4942972958087921),
 ('rockets', 0.4914822280406952),
 ('rl10', 0.491318017244339),
 ('specific', 0.4873909652233124),
 ('course', 0.48427194356918335)]

Las palabras que más se relacionan con el termo 'rocket' (cohete) son partes de cohetes (motors, boosters, strap, srb, rl10) o son conceptos relacionados con la performance (specific, course)

In [43]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["rocket"], topn=10)

[('etops', -0.02056294120848179),
 ('including', -0.043586160987615585),
 ('could', -0.0483795590698719),
 ('mid', -0.05084841698408127),
 ('\u2003orders', -0.06154000759124756),
 ('routes', -0.06309053301811218),
 ('\u2003deliveries', -0.06773940473794937),
 ('us', -0.07051333039999008),
 ('operating', -0.07271091639995575),
 ('entered', -0.07688814401626587)]

Las palabras que menos se relacionan con 'rocket' son palabras que tienen más relación con aviones (etops, routes, operating, entered) o son palabras normales (including, mid, could, orders, deliveries)

In [39]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('ten', 0.583423376083374),
 ('rl10', 0.5743970274925232),
 ('three', 0.5734760761260986),
 ('merlin', 0.573472261428833),
 ('149', 0.5675811171531677),
 ('nine', 0.5592108368873596),
 ('ssme', 0.5576633810997009),
 ('srb', 0.5564800500869751),
 ('legs', 0.5527510046958923),
 ('window', 0.5481016635894775)]

Las palabras que más se relacionan con 'four' son números (ten, three, nine) o son palabras en plural o objetos (ssme, legs, srb).

In [40]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('website', 0.8378516435623169),
 ('financial', 0.8047073483467102),
 ('responsibility', 0.8030418753623962),
 ('amount', 0.7965542078018188),
 ('council', 0.7855682373046875)]

Las palabras que más se relacionan con 'money' son palavras associadas con conceptos financeiros (amount, financial, responsibility) o son otros conceptos que también son utilizados obtención de informaciones (website), o para su administración (council)

### 5 - Visualizar agrupación de vectores

In [41]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    
    #labels = np.asarray(model.wv.index2word)  
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [42]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

Lo espacio de embeddings mostrado acima apresenta alguns agrupamientos interessantes. Los años (2017, 2014...), están muy cerca, así como los nombres de aeronaves (747, a330, a350).
Los números también formam su grupo. En la parte superior podemos ver conceptos relacionados con los cohetes.
Cerca de estos podemos ver conceptos relacionados a transporte (weight, capacity, increased, passengers)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.